# Sudoku

## 1. Problématique

<div align="middle"><h3>Peut-on créer un programme pour construire une grille complète et correcte?</h3></div>

## 2. Modélisation

<div align="middle"><img src="ressources/grille4x4.png" width=400px></div>

<div align="middle"><img src="ressources/graphe4x4.png" width=400px></div>

## 3. Implémentation

### 3.1 Classe Sudoku

#### Activité 1
- Écrire la classe *Sudoku* et son constructeur. Elle possédera trois attributs à initialiser:
    - **taille: int** 4 pour un petit Sudoku,
    - **grille: list** à construire en fonction de la figure,
    - **graphe**: Graphe

In [ ]:
class Sudoku:

    def __init__(self, taille):
        self.taille = taille
        self.grille = [[0 for _ in range(self.taille)] for _ in range(self.taille)]
        self.graphe = Graphe()

### 3.2 Créer le graphe

#### Activité 2
Écrire la méthode **creer_graphe(self)$\;\rightarrow\;$None** qui construit le graphe.

In [ ]:
    def creer_graphe(self):
        """
        Initialise le graphe des cases liées
        """
        # création des sommets
        for x in range(self.taille):
            for y in range(self.taille):
                self.graphe.ajouter_sommet((x,y))

In [ ]:
        # arêtes verticales
        for x in range(self.taille):
            for y in range(self.taille-1):
                for k in range(y+1, self.taille):
                    self.graphe.ajouter_arete((x,y),(x,k))

        # arêtes horizontales
        for y in range(self.taille):
            for x in range(self.taille-1):
                for k in range(x+1, self.taille):
                    self.graphe.ajouter_arete((x,y),(k,y))

In [ ]:
        # blocs
        nb_blocs = int(sqrt(self.taille))
        
        # il y a (nb_blocs*nb_blocs) blocs
        for i in range(nb_blocs):
            for j in range(nb_blocs):
                
                # pour chaque case
                for y in range(nb_blocs):
                    for x in range(nb_blocs):
                        
                        # vers chaque case
                        for lig in range(nb_blocs):
                            for col in range(nb_blocs):
                                
                                if not((x,y) == (col,lig)):
                                    """
                                    on ne crée pas d'arête vers lui-même
                                    par contre on ne gère pas les éventuels doublons:
                                    l'ensemble (set) dans ajouter_arete le fait pour nous
                                    """
                                    self.graphe.ajouter_arete(
                                        (j*nb_blocs + x  , i*nb_blocs + y),
                                        (j*nb_blocs + col, i*nb_blocs + lig)
                                        )

### 3.3 Méthodes utiles

#### Activité 3
Écrire la méthode **case_suivante(self, s: tuple)$\;\rightarrow\;$tuple** qui renvoie les coordonnées, sous forme d'un tuple, de la case suivant celle de coordonnées s.

In [ ]:
    def case_suivante(self, s: tuple)->tuple:
        """
        retourne la prochaine case à visiter

        Parameters
        ----------
        s : tuple
            le sommet actuel.

        Returns
        -------
        tuple
            le prochain sommet.
        """
        x,y = s[0],s[1]
        if y == (self.taille - 1): # dernière colonne
            x += 1
            y = 0
        else:
            y += 1
        return (x,y)

#### Activité 4
Écrire la méthode **est_possible(self, s: tuple, choix: int)$\;\rightarrow\;$bool** qui renvoie *True* si le chiffre *choix* n'est pas déjà positionné dans une case *adjacente* au sommet s.

In [ ]:
    def est_possible(self, s: tuple, choix: int)->bool:
        """
        vérifie si choix n'est pas déjà utilisé

        Parameters
        ----------
        choix : int
            la valeur à tester.

        Returns
        -------
        bool
        """
        for v in self.graphe.get_adjacents(s):
            if choix == self.grille[v[0]][v[1]]:
                return False
        return True

### 3.4 Remplir la grille

Algorithme:
- Si nous sortons de la grille, elle a été correctement remplie: renvoyer True.
- Sinon, pour chaque chiffre:
     - S'il peut être positionné:
         - Le placer dans la grille.
         - Remplir récursivement la case suivante et remonter True dans la pile d'appel si le placement est correct.
- # Tous les chiffres ont été testés et aucun ne fonctionne: 
- Réinitialiser la valeur de la case.
- Remonter False dans la pile d'appel.

<div align="middle"><img src="ressources/algo01.png" width=300px></div>

- On sélectionne et place 3.
- On effectue un appel récursif sur la case suivante.

<div align="middle"><img src="ressources/algo02.png" width=300px></div>

- On teste 1 et 3: ils ne sont pas positionnables.

<div align="middle"><img src="ressources/algo03.png" width=300px></div>

- On sélectionne et place 4.
- On effectue un appel récursif sur la case suivante.

<div align="middle"><img src="ressources/algo04.png" width=300px></div>

- On teste toutes les valeurs: aucune n'est possible.
- (On efface cette case.)
- On renvoie False: la valeur placée n'est pas correcte.

<div align="middle"><img src="ressources/algo05.png" width=300px></div>

- On sélectionne et place 2.
- On effectue un appel récursif sur la case suivante.

<div align="middle"><img src="ressources/algo06.png" width=300px></div>

- On teste toutes les valeurs: aucune n'est possible.
- (On efface cette case.)
- On renvoie False: la valeur placée n'est pas correcte.

<div align="middle"><img src="ressources/algo07.png" width=300px></div>

- Toutes les valeurs ont été testées: aucune n'est possible.
- On efface cette case.
- On renvoie False: la valeur placée n'est pas correcte.

#### Activité 5
Écrire la méthode **remplir_rec(self, s: tuple = (0,0))$\;\rightarrow\;$None** qui implémente cet algorithme.

In [ ]:
def remplir_rec(self, s: tuple = (0,0)):
        # si on a tout rempli = on sort du tableau
        if s[0] == self.taille:
            return True

        # On ajoute un peu d'aléatoire dans le choix
        choix = list(range(1,self.taille+1))
        shuffle(choix)
        # test des valeurs possibles
        for val in choix:
            # regarde si choix n'est pas déjà dans les voisins
            if self.est_possible(s, val):
                self.grille[s[0]][s[1]] = val
                if self.remplir_rec(self.case_suivante(s)):
                    return True

        # Tous les chiffres ont été testés et aucun ne fonctionne
        # on réinitialise alors la case
        self.grille[s[0]][s[1]] = 0
        return False

## 4. Affichage
#### Activité 6
Écrire la méthode **afficher(self)$\;\rightarrow\;$None** qui utilise la bibliothèque *tkinter* pour réaliser un affichage de la grille.

In [ ]:
def afficher(self):
        taille_case = 100
        self.fenetre = Tk()
        self.fenetre.title('Sudoku')
        self.canevas = Canvas(self.fenetre,width=taille_case*self.taille,
                                          height=taille_case*self.taille)
        self.canevas.pack()

In [ ]:
    for l in range(self.taille):
            for c in range(self.taille):

In [ ]:
                # ligne verticale
                if c%sqrt(self.taille) == 0:
                    trait_vert = 3
                else:
                    trait_vert = 1

                self.canevas.create_line(taille_case*c,
                                     0,
                                     taille_case*c,
                                     taille_case*self.taille,
                                     width=trait_vert)

In [ ]:
                # ligne horizontale
                if l%sqrt(self.taille) == 0:
                    trait_hori = 3
                else:
                    trait_hori = 1
                self.canevas.create_line(0,
                                         taille_case*l,
                                         taille_case*self.taille,
                                         taille_case*l,
                                         width=trait_hori)

In [ ]:
                # chiffre
                self.canevas.create_text(taille_case*c + taille_case//2,
                                         taille_case*l + taille_case//2,
                                         text=str(self.grille[l][c]),
                                         font="Arial "+str(taille_case//2))

In [ ]:
        self.fenetre.mainloop()